In [2]:
# import tensorflow and numpy
import tensorflow as tf
import numpy as np

In [3]:
gradients = 5
vals = 3
iterations = 20000
learning_rate = 0.01
ex = gradients

def training_data():
    n = 0
    x = np.arange(vals+1).astype(np.int32)
    y = np.array([])
    for i in range(gradients):
        for e in range(ex):
            y = np.append(y, x*n+e).reshape((e)+(i*gradients+1),vals+1)

        n+=1
        
    return(y,np.size(y,0))

print(training_data())

(array([[  0.,   0.,   0.,   0.],
       [  1.,   1.,   1.,   1.],
       [  2.,   2.,   2.,   2.],
       [  3.,   3.,   3.,   3.],
       [  4.,   4.,   4.,   4.],
       [  0.,   1.,   2.,   3.],
       [  1.,   2.,   3.,   4.],
       [  2.,   3.,   4.,   5.],
       [  3.,   4.,   5.,   6.],
       [  4.,   5.,   6.,   7.],
       [  0.,   2.,   4.,   6.],
       [  1.,   3.,   5.,   7.],
       [  2.,   4.,   6.,   8.],
       [  3.,   5.,   7.,   9.],
       [  4.,   6.,   8.,  10.],
       [  0.,   3.,   6.,   9.],
       [  1.,   4.,   7.,  10.],
       [  2.,   5.,   8.,  11.],
       [  3.,   6.,   9.,  12.],
       [  4.,   7.,  10.,  13.],
       [  0.,   4.,   8.,  12.],
       [  1.,   5.,   9.,  13.],
       [  2.,   6.,  10.,  14.],
       [  3.,   7.,  11.,  15.],
       [  4.,   8.,  12.,  16.]]), 25)


In [4]:
data,training_lines = training_data()
length = (gradients-1)*(vals+1)+1
full_length = length*vals
def set_data():
    in_data = np.zeros([training_lines,vals,length])
    lab = np.zeros([training_lines,1,length])
    for i in range(training_lines):
        for a in range(vals):
            in_data[i][a][data[i][a]] = 1
        lab[i][0][data[i][vals]] = 1
    in_data = in_data.reshape(training_lines,1,full_length)
    return(in_data,lab)    

In [5]:
x = tf.placeholder(tf.float32, [None, full_length])
W = tf.Variable(tf.zeros([full_length, length]))
b = tf.Variable(tf.zeros([length]))
y = tf.matmul(x, W) + b

y_ = tf.placeholder(tf.float32, [None, length])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [6]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)

sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options)) 

init = tf.initialize_all_variables()
sess.run(init)
                
for _ in range(iterations):

    x_data, y_data = set_data()
    x_data = x_data[_//(iterations/(training_lines-1))]
    y_data = y_data[_//(iterations/(training_lines-1))]
    
    sess.run(train_step, feed_dict={x: x_data, y_: y_data})

    if _ % 1000 == 0:
        print(_)
        print(sess.run(cross_entropy, feed_dict={x: x_data, y_: y_data}))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0
2.79561
1000
0.272871
2000
0.10301
3000
0.061422
4000
0.0434086
5000
3.09928
6000
0.324862
7000
0.104249
8000
0.0615834
9000
0.040009
10000
4.97567
11000
0.330349
12000
0.107215
13000
0.0563794
14000
0.0392178
15000
4.63474
16000
0.326812
17000
0.0938911
18000
0.0550789
19000
0.0429186


In [11]:
test_line = np.array([])
for i in range(vals):
    val = int(input(('value',i,'?')))
    test_line = np.append(test_line, val)
def test_model():
    input_array = np.array([])
    for a in range(vals):
        test_array = np.zeros([1,length])
        test_array[0][test_line[a]] = 1
        input_array = np.append(input_array, test_array).reshape([1,(a+1)*length])
    return(input_array)        
test_model()

('value', 0, '?')3
('value', 1, '?')6
('value', 2, '?')9


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [12]:
probs = sess.run(y, feed_dict={x:test_model()})
print(probs)
print "Learnt answer =", np.argmax(probs)
print 'Probaility of', np.argmax(probs), '=', probs[0][np.argmax(probs)]
answer = (test_line[1]-test_line[0])+test_line[vals-1]
print "numerical answer =", answer
print 'probability of', answer, '=', probs[0][answer]

[[-0.9925347  -1.07847381 -1.18294406 -0.0151965  -1.48708689 -1.05759811
   0.06485161 -1.77753687 -0.99015093  0.18448338 -0.03121322 -0.98118514
   4.50694656  1.18209553  2.32715368  2.4466424  -1.1182456 ]]
Learnt answer = 12
Probaility of 12 = 4.50695
numerical answer = 12.0
probability of 12.0 = 4.50695


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
